# WA - King ilçesi fiyat tahmini

In [2]:
import pandas as pd
pd.set_option('display.max_columns',55)
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score , mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge , Lasso , ElasticNet
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, PowerTransformer

In [3]:
df=pd.read_csv('kc_house_data.csv')

In [4]:
df.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [5]:
df.corr(numeric_only=True)

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,1.000000,-0.016762,0.001286,0.005160,-0.012258,-0.132109,0.018525,-0.002721,0.011592,-0.023783,0.008130,-0.010842,-0.005151,0.021380,-0.016907,-0.008224,-0.001891,0.020799,-0.002901,-0.138798
price,-0.016762,1.000000,0.308350,0.525138,0.702035,0.089661,0.256794,0.266369,0.397293,0.036362,0.667434,0.605567,0.323816,0.054012,0.126434,-0.053203,0.307003,0.021626,0.585379,0.082447
bedrooms,0.001286,0.308350,1.000000,0.515884,0.576671,0.031703,0.175429,-0.006582,0.079532,0.028472,0.356967,0.477600,0.303093,0.154178,0.018841,-0.152668,-0.008931,0.129473,0.391638,0.029244
bathrooms,0.005160,0.525138,0.515884,1.000000,0.754665,0.087740,0.500653,0.063744,0.187737,-0.124982,0.664983,0.685342,0.283770,0.506019,0.050739,-0.203866,0.024573,0.223042,0.568634,0.087175
sqft_living,-0.012258,0.702035,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286
sqft_lot,-0.132109,0.089661,0.031703,0.087740,0.172826,1.000000,-0.005201,0.021604,0.074710,-0.008958,0.113621,0.183512,0.015286,0.053080,0.007644,-0.129574,-0.085683,0.229521,0.144608,0.718557
floors,0.018525,0.256794,0.175429,0.500653,0.353949,-0.005201,1.000000,0.023698,0.029444,-0.263768,0.458183,0.523885,-0.245705,0.489319,0.006338,-0.059121,0.049614,0.125419,0.279885,-0.011269
waterfront,-0.002721,0.266369,-0.006582,0.063744,0.103818,0.021604,0.023698,1.000000,0.401857,0.016653,0.082775,0.072075,0.080588,-0.026161,0.092885,0.030285,-0.014274,-0.041910,0.086463,0.030703
view,0.011592,0.397293,0.079532,0.187737,0.284611,0.074710,0.029444,0.401857,1.000000,0.045990,0.251321,0.167649,0.276947,-0.053440,0.103917,0.084827,0.006157,-0.078400,0.280439,0.072575
condition,-0.023783,0.036362,0.028472,-0.124982,-0.058753,-0.008958,-0.263768,0.016653,0.045990,1.000000,-0.144674,-0.158214,0.174105,-0.361417,-0.060618,0.003026,-0.014941,-0.106500,-0.092824,-0.003406


In [6]:
abs(df.corr(numeric_only=True))['price'].sort_values(ascending=False)

price            1.000000
sqft_living      0.702035
grade            0.667434
sqft_above       0.605567
sqft_living15    0.585379
bathrooms        0.525138
view             0.397293
sqft_basement    0.323816
bedrooms         0.308350
lat              0.307003
waterfront       0.266369
floors           0.256794
yr_renovated     0.126434
sqft_lot         0.089661
sqft_lot15       0.082447
yr_built         0.054012
zipcode          0.053203
condition        0.036362
long             0.021626
id               0.016762
Name: price, dtype: float64

In [7]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [8]:
# Log dönüşümleri
df['log_price'] = np.log1p(df['price'])
df['log_sqft_living'] = np.log1p(df['sqft_living'])
df['log_sqft_lot'] = np.log1p(df['sqft_lot'])
df['log_sqft_above'] = np.log1p(df['sqft_above'])
df['log_sqft_basement'] = np.log1p(df['sqft_basement'])

# Yeni özellikler
df['price_per_sqft'] = df['price'] / df['sqft_living']
df['basement_ratio'] = df['sqft_basement'] / df['sqft_living']
df['living_lot_ratio'] = df['sqft_living'] / df['sqft_lot']
df['has_basement'] = (df['sqft_basement'] > 0).astype(int)
df['has_renovation'] = (df['yr_renovated'] > 0).astype(int)
df['total_rooms'] = df['bedrooms'] * df['bathrooms']

# Etkileşim özellikleri
df['grade_condition'] = df['grade'] * df['condition']
df['living_grade'] = df['sqft_living'] * df['grade']
df['location_grade'] = df['grade'] * (df['lat'] + df['long'])

In [9]:
df.shape

(21613, 35)

In [10]:
def remove_outliers_iqr(df, columns):
    Q1 = df[columns].quantile(0.25)
    Q3 = df[columns].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_clean = df[~((df[columns] < lower_bound) | (df[columns] > upper_bound)).any(axis=1)]
    return df_clean

# Outlier'ları temizleyeceğimiz sayısal sütunlar
numerical_columns = ["price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "sqft_above", "sqft_basement"]
df_clean = remove_outliers_iqr(df, numerical_columns)

In [11]:
df.shape

(21613, 35)

In [12]:
df['age']=2024-df['yr_built']

In [13]:
df['age'] = np.where(df['yr_renovated'] > 0, 2024 - df['yr_renovated'], df['age'])

In [14]:
df['zipcode']=df['zipcode'].astype('category')

In [15]:
df['bedrooms'] = np.sqrt(df['bedrooms'])
df['bathrooms'] = np.sqrt(df['bathrooms'])

In [16]:
df['sqft_living_2'] = df['sqft_living'] ** 20
df['grade_2'] = df['grade'] ** 20
df['bathrooms_2'] = df['bathrooms'] ** 20

In [17]:
x = df[[
    'log_sqft_living', 'grade', 'log_sqft_above', 'sqft_living15',
    'bathrooms', 'view', 'waterfront', 'floors', 'bedrooms',
    'price_per_sqft', 'basement_ratio', 'living_lot_ratio',
    'has_basement', 'has_renovation', 'total_rooms', 'age',
    'grade_condition', 'living_grade', 'location_grade'
]]

In [18]:
y = df['log_price']  # Log-dönüşümlü fiyatı hedef değişken olarak kullan

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [20]:
rf_model = RandomForestRegressor(
    n_estimators=200,  # Ağaç sayısını artır
    max_depth=15,      # Maksimum derinliği sınırla
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    n_jobs=-1,         # Paralel işlem
    random_state=42
)
rf_model.fit(x_train, y_train)

RandomForestRegressor(max_depth=15, max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=5, n_estimators=200, n_jobs=-1,
                      random_state=42)

In [21]:
y_pred = rf_model.predict(x_test)

# Log ölçeğinden geri dönüştür
y_test_exp = np.expm1(y_test)
y_pred_exp = np.expm1(y_pred)

# Metrikleri hesapla
r2 = r2_score(y_test_exp, y_pred_exp)
rmse = np.sqrt(mean_squared_error(y_test_exp, y_pred_exp))

In [22]:
r2, rmse

(0.954029261469835, 83364.89135727311)

en başta elimden geleni yaptım 4 5 farklı dosyada teker teker grafikleri felan çizdirdim ama hep birbirine girdi en son işin içinden çıkamadım gpt den yardım aldım ondan da denediğim kısımları sildim sadece kodun asıl kısımları kaldı ama konu hala tam oturmadı asistan hocalarımla yarın gün içinde konuşup daha derin öğreneceğimi düşünüyorum bunu biraz ödev zamanı geçmeden teslim etmek için atıyorum umarım anlayışla karşılarsınız 